# US Walkability


This project is to show walkability of the walk score data sample provided by walkscore.com of Washington D.C. 

![](score.PNG)

More information can be found at https://www.walkscore.com/how-it-works/

In [2]:
import pandas as pd

In [28]:
url = "https://docs.google.com/spreadsheets/u/0/d/1hQHrdwGrYmWjuRuCxQ-hmCr51jgZlJ7YLG5y1F1VExg/pub?output=html"

data = pd.read_html(url, header=1)
data = data[0]
print(data.shape)
data.head()


(65, 23)


,1,Snapped lat,Snapped lon,Walk Score,Walk Score Label,Average Block Length (meters),Average Block Length Score,Intersection Density (inscts/mi^2),Intersection Density Score,Transit Score,...,City Name,City Walk Score,Rail Stops in 0.5mi,Errands Score,Culture Score,Grocery Score,Park Score,Dining and Drinking Score,School Score,Shopping Score
0,2,38.9010,-77.0460,99,Walker's Paradise,104.18,95.82,250.83,83.61,95,...,Washington D.C.,73.2,3,100.00,93.45,97.89,100.0,100.00,99.67,99.63
1,3,38.9115,-77.0355,97,Walker's Paradise,97.10,100.00,287.75,95.92,92,...,Washington D.C.,73.2,1,97.70,78.34,100.00,100.0,99.50,94.49,98.15
2,4,38.9040,-77.0250,98,Walker's Paradise,100.95,99.05,247.01,82.34,100,...,Washington D.C.,73.2,5,99.98,97.27,95.08,100.0,99.77,97.74,95.49
3,5,38.8995,-77.0385,99,Walker's Paradise,120.64,79.36,184.62,61.54,98,...,Washington D.C.,73.2,3,100.00,94.35,98.72,100.0,99.98,100.00,96.78
4,6,38.9100,-77.0475,100,Walker's Paradise,98.83,100.00,255.92,85.31,85,...,Washington D.C.,73.2,1,98.91,94.89,100.00,100.0,100.00,100.00,99.20
